Connect to the Swiss Open Data API to retrieve the population Data by age for the Canton of Zurich. Afterwards load the Data into a Dataframe and save the originally retrieved data into csv, before the cleaning of the Data.

In [1]:
import requests    
import json         
import pandas as pd 
import os

package = 'bevolkerung-nach-gemeinde-heimat-geschlecht-und-alter'

# Base url for the open data swiss API
base_url = 'https://opendata.swiss/api/3/action/package_show?id='

# Construct the url including package 
package_information_url = base_url + package

# HTTP request
package_information = requests.get(package_information_url)

# Use the json module to load CKAN's response into a dictionary
package_dict = json.loads(package_information.content)

# Check the contents of the response.
assert package_dict['success'] is True  
package_dict = package_dict['result']   
print(package_dict)             

KeyboardInterrupt: 

In [ ]:
# Get the url for the data from the dictionary
data_url = package_dict['resources'][0]['url']
print('Data url:' + data_url)

# Print the data format
data_format = package_dict['resources'][0]['format']
print('Data format:' + data_format)

Data url:https://www.web.statistik.zh.ch/ogd/daten/ressourcen/KTZH_00000254_00001282.csv
Data format:XLS


In [ ]:
# Within the dictionary the data is not correctly marked as csv thats why we are saying xls but reading it as csv
xls = ['XLS']
if any(s in data_format for s in xls):
    BevoelkerungAlterdf = pd.read_csv(data_url,sep=';')
else:
    print('Sorry, the data format is not supported')
BevoelkerungAlterdf

,jahr,gemeinde_bfs_nr,gemeinde,bezirk_code,bezirk,region_code,region_name,heimat_code,heimat,geschlecht_code,geschlecht,einjahresaltersklasse,anzahl
0,2010,1,Aeugst am Albis,101,Affoltern,105,Knonaueramt,1,Schweiz,1,Mann,0,8
1,2010,1,Aeugst am Albis,101,Affoltern,105,Knonaueramt,1,Schweiz,1,Mann,1,9
2,2010,1,Aeugst am Albis,101,Affoltern,105,Knonaueramt,1,Schweiz,1,Mann,2,7
3,2010,1,Aeugst am Albis,101,Affoltern,105,Knonaueramt,1,Schweiz,1,Mann,3,13
4,2010,1,Aeugst am Albis,101,Affoltern,105,Knonaueramt,1,Schweiz,1,Mann,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
736184,2023,298,Wiesendangen,110,Winterthur,109,Winterthur und Umgebung,2,Ausland,2,Frau,84,1
736185,2023,298,Wiesendangen,110,Winterthur,109,Winterthur und Umgebung,2,Ausland,2,Frau,85,2
736186,2023,298,Wiesendangen,110,Winterthur,109,Winterthur und Umgebung,2,Ausland,2,Frau,87,1
736187,2023,298,Wiesendangen,110,Winterthur,109,Winterthur und Umgebung,2,Ausland,2,Frau,88,1


In [ ]:
#Save original data files to csv, if the folder is not aleady created, it will be created
directory = 'Original_CSV_files'
current_directory = os.getcwd()

csv_path = os.path.join(current_directory, directory, 'BevoelkerungnachAlter_KantonZurich_2009_2023.csv')

# Create the directory if it doesn't exist
if not os.path.exists(os.path.join(current_directory, directory)):
    os.makedirs(os.path.join(current_directory, directory))

BevoelkerungAlterdf.to_csv(csv_path, index=False)
print(BevoelkerungAlterdf)

Drop not needed Columns, Filter the Data to the years 2009-2023, Delete the summarized Datarows an as well unify the Region Names so the data can be analyzed with the other available datasets. As well include Age Groups so it is easier to work with the data  and reduce the amount of data rows. Afterwards save a Cleaned csv file to the Cleaned CSV Files folder. 

In [ ]:
BevoelkerungAlterdf.drop(['bezirk_code','bezirk','region_code','region_name','heimat_code','geschlecht_code','geschlecht','heimat','gemeinde_bfs_nr'], axis=1, inplace=True)
BevoelkerungAlterdf

,jahr,gemeinde,einjahresaltersklasse,anzahl
0,2010,Aeugst am Albis,0,8
1,2010,Aeugst am Albis,1,9
2,2010,Aeugst am Albis,2,7
3,2010,Aeugst am Albis,3,13
4,2010,Aeugst am Albis,4,5
...,...,...,...,...
736184,2023,Wiesendangen,84,1
736185,2023,Wiesendangen,85,2
736186,2023,Wiesendangen,87,1
736187,2023,Wiesendangen,88,1


In [ ]:
# Vereinheitlichen der Gemeindenamen
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Aesch (ZH)', 'Aesch', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Benken (ZH)', 'Benken', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Birmensdorf (ZH)', 'Birmensdorf', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Buchs (ZH)', 'Buchs', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Erlenbach (ZH)', 'Erlenbach', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Gossau (ZH)', 'Gossau', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Kilchberg (ZH)', 'Kilchberg', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Küsnacht (ZH)', 'Küsnacht', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Rickenbach (ZH)', 'Rickenbach', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Rüti (ZH)', 'Rüti', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Schlatt (ZH)', 'Schlatt', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Wald (ZH)', 'Wald', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Weiningen (ZH)', 'Weiningen', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Wetzikon (ZH)', 'Wetzikon', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Wil (ZH)', 'Wil', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Zell (ZH)', 'Zell', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Aeugst am Albis', 'Aeugst', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Affoltern am Albis', 'Affoltern', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Berg am Irchel', 'Berg', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Buch am Irchel', 'Buch', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Ellikon an der Thur', 'Ellikon', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Hausen am Albis', 'Hausen', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Kappel am Albis', 'Kappel', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Langnau am Albis', 'Langnau', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Oetwil am See', 'Oetwil', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Thalheim an der Thur', 'Thalheim', regex=False)
BevoelkerungAlterdf['gemeinde'] = BevoelkerungAlterdf['gemeinde'].str.replace('Uetikon am See', 'Uetikon', regex=False)


BevoelkerungAlterdf

,jahr,gemeinde,einjahresaltersklasse,anzahl
0,2010,Aeugst,0,8
1,2010,Aeugst,1,9
2,2010,Aeugst,2,7
3,2010,Aeugst,3,13
4,2010,Aeugst,4,5
...,...,...,...,...
736184,2023,Wiesendangen,84,1
736185,2023,Wiesendangen,85,2
736186,2023,Wiesendangen,87,1
736187,2023,Wiesendangen,88,1


In [ ]:
import pandas as pd

altersklassen = {
    '0-14 Jahre alt': range(0, 15),
    '15-29 Jahre alt': range(15, 30),
    '30-44 Jahre alt': range(30, 45),
    '45-59 Jahre alt': range(45, 60),
    '60-74 Jahre alt': range(60, 75),
    '75-89 Jahre alt': range(75, 90),
    '90-104 Jahre alt': range(90, 105)
}

# Funktion zur Aggregation der Altersklassen
def summiere_altersklassen(gruppe):
    summen = {}
    for altersklasse, bereich in altersklassen.items():
        summen[altersklasse] = gruppe[gruppe['einjahresaltersklasse'].isin(bereich)]['anzahl'].sum()
    return pd.Series(summen, dtype='int64')

# Gruppiere den DataFrame nach Gemeinde und Jahr und wende die Funktion auf jede Gruppe an
summierteBevoelkerungAlter_df = BevoelkerungAlterdf.groupby(['gemeinde', 'jahr'], as_index=False).apply(summiere_altersklassen)

# Ergebnis anzeigen
print(summierteBevoelkerungAlter_df)


      gemeinde  jahr  0-14 Jahre alt  15-29 Jahre alt  30-44 Jahre alt  \
0     Adliswil  2010            2348             2907             3778   
1     Adliswil  2011            2510             3087             4130   
2     Adliswil  2012            2582             3159             4367   
3     Adliswil  2013            2673             3060             4476   
4     Adliswil  2014            2786             2993             4503   
...        ...   ...             ...              ...              ...   
2235    Zürich  2019           58927            72875           125728   
2236    Zürich  2020           59578            72433           126541   
2237    Zürich  2021           59876            73173           126052   
2238    Zürich  2022           59694            75261           127463   
2239    Zürich  2023           59958            76773           128838   

      45-59 Jahre alt  60-74 Jahre alt  75-89 Jahre alt  90-104 Jahre alt  
0                3389             2

/var/folders/lt/9lkssqds44vcn4hwlh7dd2wc0000gn/T/ipykernel_7325/1620022716.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summierte_df = df.groupby(['gemeinde', 'jahr'], as_index=False).apply(summiere_altersklassen)


In [ ]:
directory = 'Cleaned_CSV_files'
current_directory = os.getcwd()

csv_path = os.path.join(current_directory, directory, 'BevoelkerungnachAlter_KantonZurich_2009_2023_Cleaned.csv')

# Create the directory if it doesn't exist
if not os.path.exists(os.path.join(current_directory, directory)):
    os.makedirs(os.path.join(current_directory, directory))

summierteBevoelkerungAlter_df.to_csv(csv_path, index=False)
print(summierteBevoelkerungAlter_df)

The DataFrame has been saved to '/Users/natalievintonjak/Documents/Data Science Project/Bevoelkerung_KantonZurich_2010_2023.csv'.


Print the Information for the Dataframe so we can create the Table for the Database correspondingly.

In [ ]:
summierteBevoelkerungAlter_df.info()

Connect to the database and create a new table called Bevoelkerungsdichte. If the table is already existing drop it.

In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

# MySQL connection parameters
host = 'localhost'
user = 'admin'
password = 'Criminal1234'
database = 'CriminalDataDB'  
try:
    connection = mysql.connector.connect(host=host,
                                         user=user,
                                         password=password,
                                         database=database)

    if connection.is_connected():
        print("Connected to MySQL server")
        cursor = connection.cursor()

        # SQL command to drop the table if it already exists becuase before that line of code the data was already inserted multiple times
        drop_table_query = "DROP TABLE IF EXISTS Bevoelkerungsdaten;"
        cursor.execute(drop_table_query)
        print("Table 'Bevoelkerungsdaten' dropped if it existed alreadyy")

        create_table_query = """
        CREATE TABLE IF NOT EXISTS Bevoelkerungsdaten (
            BFS_NR INT,
            GEBIET_NAME VARCHAR(255),
            INDIKATOR_NAME VARCHAR(255),
            INDIKATOR_JAHR INT,
            INDIKATOR_VALUE FLOAT,
            EINHEIT_LANG VARCHAR(255)
        )
        """

        cursor.execute(create_table_query)
        print("Table 'Bevoelkerungsdaten' created successfully")

except Error as e:
    print("Error connecting to MySQL:", e)

finally:
    if connection.is_connected():
        # Close cursor and connection
        cursor.close()
        connection.close()
        print("MySQL connection closed")


Insert the Cleanaed data from the Cleaned Csv file which was created before. 

In [ ]:
import mysql.connector 
from mysql.connector import Error
import pandas as pd

# MySQL connection parameters
connection_params = {
    'host': 'localhost',
    'user': 'admin',
    'password': 'Criminal1234',
    'database': 'CriminalDataDB',
    'allow_local_infile': True
}

directory = 'Cleaned_CSV_files'
current_directory = os.getcwd()
csv_file_path = os.path.join(current_directory, directory, 'BevoelkerungnachAlter_KantonZurich_2009_2023_Cleaned.csv')

try:
    with mysql.connector.connect(**connection_params) as connection:
        print("Connected to MySQL server")

        Bevoelkerungsdichte_df_to_sql = pd.read_csv(csv_file_path)

        cursor = connection.cursor()
        insert_query = """
        LOAD DATA LOCAL INFILE %s 
        INTO TABLE Bevoelkerungsdaten 
        FIELDS TERMINATED BY ',' 
        ENCLOSED BY '"' 
        LINES TERMINATED BY '\n' 
        IGNORE 1 LINES
        """

        cursor.execute(insert_query, (csv_file_path,))
        connection.commit()

        print("Data from CSV file successfully inserted into MySQL table 'Bevoelkerungsdaten'")

except Error as e:
    print("Error connecting to MySQL:", e)

finally:
    if connection.is_connected():
        # Close cursor and connection
        cursor.close()
        connection.close()
        print("MySQL connection closed")


Check if the Data upload to the table was successfully by creating a query to the sql table. 

In [ ]:
import mysql.connector
from mysql.connector import Error

# MySQL connection parameters
connection_params = {
    'host': 'localhost',
    'user': 'admin',
    'password': 'Criminal1234',
    'database': 'CriminalDataDB'
}

try:
    connection = mysql.connector.connect(**connection_params)
    print("Connected to MySQL server")

    with connection.cursor() as cursor:
        select_query = "SELECT * FROM Bevoelkerungsdaten"

        cursor.execute(select_query)
        rows = cursor.fetchall()

        for row in rows:
            print(row)

except Error as e:
    print("Error connecting to MySQL:", e)

finally:
    if 'connection' in locals() and connection.is_connected():
        # Close connection
        connection.close()
        print("MySQL connection closed")